In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import glob
import math

In [ ]:
# function to read in frame images
def load_images_from_folder(frames):
    images = []
    for filename in os.listdir(frames):
        img = cv2.imread(os.path.join(frames,filename))
        if img is not None:
            images.append(img)
    return images

# print(load_images_from_folder(frames))

In [ ]:
# sets the directory
directory = '/Users/eoinmurnaghan/Desktop/video_post_processing/data/acceleration/'
output_directory = '/Users/eoinmurnaghan/Desktop/video_post_processing/direction_outputs/acceleration_max/'

# adds each of the filenames within the data folder to the files list
files = os.listdir(directory)

# print(files)

In [ ]:
# read in box coordinate data
box_data = pd.read_csv('/Users/eoinmurnaghan/Desktop/video_post_processing/log_data_cleaning/output_acceleration.csv')

box_data

In [ ]:
# getFrameNumber function
def getFrameNumber(files):
    
    # empty frameNumbers list    
    frameNumbers = []
    
    for file in range(len(files) - 2):
        
        # appends each frameNumber to the list
        frameNumbers.append(file + 1)
        
    # returns the framNumbers list     
    return frameNumbers
# end of getFrameNumber function

In [ ]:
# box_data['Frame'].value_counts()
box_data['freq'] = box_data.groupby('Frame')['Frame'].transform('count')

box_data.head()

In [ ]:
# rename columns to reflect data
box_data.rename(columns={'ymin.1':'xmin','ymin.2':'ymax','ymin.3':'xmax'}, inplace=True)
box_data.rename(columns={'ymin2.1':'xmin2','ymin2.2':'ymax2','ymin2.3':'xmax2',}, inplace=True)
box_data.rename(columns={'ymin3.1':'xmin3','ymin3.2':'ymax3','ymin3.3':'xmax3',}, inplace=True)
box_data.rename(columns={'ymin4.1':'xmin4','ymin4.2':'ymax4','ymin4.3':'xmax4',}, inplace=True)

# display box_data
box_data

In [ ]:
# define frame size
xsize = 640
ysize = 480

#identify vehicle in frame - (320,300) for Acceleration, (300,300) for Endurance
car_x = 320
car_y = 300

# define rhs and lhs coords for field of vision
rhs = 500
lhs = 140

# define straight ahead coorinfates - (320, 200) for Acceleration, (300, 200) for Endurance
straight_x = 320
straight_y = 200

In [ ]:
# empty frameNumbers list    
frameNumbers = []
    
for file in range(len(files) - 2):
        
    # appends each frameNumber to the list
    frameNumbers.append(file + 1)

#initialise command dataframe
df = pd.DataFrame()
df2 = pd.DataFrame()

# for loop that iterates through each frame in the frameNumbers list
for frame in frameNumbers:
    
    # nested for loop   
    for index, row in box_data.iterrows():
    
        if (row['Frame'] == frame):
        
            # casted frame number to string and then read in image data 
            image = cv2.imread(directory+"frame"+str(frame)+".jpg")
            
            
            
            
        
            # logic for 1 or no cones detected      
            if (np.isnan(row['ymin2'])):
                
                if (row['xmin'] >= 0 and row['ymin'] >= 0):
                    
                    # if cone detected is on the rhs of frame
                    if (row['xmin'] >= 0.5):
                        
                        # convert box coordinates
                        ymin1 = row['ymin'] * ysize
                        xmin1 = row['xmin'] * xsize
                        
                        # calculate midpoint between box and LHS of frame
                        midpoint_x = int(lhs + (0.5*(xmin1)))
                        midpoint_y = int(ymin1)
                          
                    else:
                        
                        #cone detected on lhs of frame
                        #convert box coordinates
                        ymin1 = row['ymin'] * ysize
                        xmin1 = row['xmin'] * xsize
                        
                        #calculate midpoint between box and RHS of frame
                        midpoint_x = int(rhs - (0.5*(xmin1)))
                        midpoint_y = int(ymin1)
                else:
                    
                    midpoint_x = straight_x
                    midpoint_y = straight_y
            
            
            
            
            
            # logic for 2 cones detected
            elif  (row['ymin2'] >= 0 and np.isnan(row['ymin3'])):
                
                # if both cones are on rhs side
                if (row['xmin'] >= 0.5 and row['xmin2'] >= 0.5):
                    
                    #calculate box closest to vehicle in horizontal direction
                    max_y = max(row['ymin'], row['ymin2'])
                    min_x = min(row['xmin'], row['xmin2'])
                    
                    #draw lines between average coords and lhs of frame
                    ymin1 = max_y * ysize
                    xmin1 = min_x * xsize
                    
                    # calculate midpoint between box and LHS of frame
                    midpoint_x = int(lhs + (0.5*(xmin1)))
                    midpoint_y = int(ymin1)
                
                #if both cones are on lhs
                elif (row['xmin'] <= 0.5 and row['xmin2'] <=0.5):
                    
                    #cones detected on lhs of frame
                    max_y = max(row['ymin'], row['ymin2'])
                    max_x = max(row['xmin'], row['xmin2'])
                    
                    #convert box coordinates
                    ymin1 = max_y * ysize
                    xmin1 = max_x * xsize
                        
                    #calculate midpoint between box1 and RHS of frame
                    midpoint_x = int(rhs - (0.5*(xmin1)))
                    midpoint_y = int(ymin1)
                
                else:
                    # convert box 1 coordinates
                    ymin1 = row['ymin'] * ysize
                    xmin1 = row['xmin'] * xsize
                
                    # convert box 2 coordinates
                    ymin2 = row['ymin2'] * ysize
                    xmin2 = row['xmin2'] * xsize

                    # midpoint calculation
                    midpoint_x = int(xmin1 + (0.5*(xmin2-xmin1)))
                    midpoint_y = int(ymin1 + (0.5*(ymin2-ymin1)))
                    
                    
        
                
            #logic for 3 cones detected
            elif  (row['ymin3'] >= 0 and np.isnan(row['ymin4'])):
                
                # box1 on lhs, box2 and box3 on rhs
                if (row['xmin'] <= 0.5 and row['xmin2'] >= 0.5 and row['xmin3'] >= 0.5):
                    
                    # LHS: convert box1 coordinates
                    ymin1 = row['ymin'] * ysize
                    xmin1 = row['xmin'] * xsize
                    
                    # RHS: boxes on rhs
                    max_y = (max(row['ymin2'], row['ymin3'])) * ysize
                    min_x = (min(row['xmin2'], row['xmin3'])) * xsize
                    
                    #midpoint calculation
                    midpoint_x = int(xmin1 + 0.5*(min_x-xmin1))
                    midpoint_y = int(ymin1 + 0.5*(max_y-ymin1))
                
                
                # box2 on lhs, box1 and box3 on rhs
                elif (row['xmin2'] <= 0.5 and row['xmin'] >= 0.5 and row['xmin3'] >= 0.5):
                    
                    # LHS: onvert box1 coordinates
                    ymin1 = row['ymin2'] * ysize
                    xmin1 = row['xmin2'] * xsize
                    
                    # RHS: boxes on rhs
                    max_y = (max(row['ymin'], row['ymin3'])) * ysize
                    min_x = (min(row['xmin'], row['xmin3'])) * xsize
                    
                    #midpoint calculation
                    midpoint_x = int(xmin1 + 0.5*(min_x-xmin1))
                    midpoint_y = int(ymin1 + 0.5*(max_y-ymin1))
        
                
                # box3 on lhs, box1 and box2 on rhs
                elif (row['xmin3'] <= 0.5 and row['xmin'] >= 0.5 and row['xmin2'] >= 0.5):
                    
                    # LHS: convert box1 coordinates
                    ymin1 = row['ymin3'] * ysize
                    xmin1 = row['xmin3'] * xsize
                    
                    # RHS: boxes on rhs
                    max_y = (max(row['ymin'], row['ymin2'])) * ysize
                    min_x = (min(row['xmin'], row['xmin2'])) * xsize
                    
                    #midpoint calculation
                    midpoint_x = int(xmin1 + 0.5*(min_x-xmin1))
                    midpoint_y = int(ymin1 + 0.5*(max_y-ymin1))
                    
        
                # box1 on rhs, box2 and box3 on lhs
                elif (row['xmin2'] <= 0.5 and row['xmin3'] <= 0.5 and row['xmin'] >= 0.5):
                    
                    # LHS: boxes on lhs
                    max_y = (max(row['ymin2'], row['ymin3'])) * ysize
                    max_x = (max(row['xmin2'], row['xmin3'])) * xsize
                    
                    # RHS: convert box1 coorindates
                    ymin1 = row['ymin'] * ysize
                    xmin1 = row['xmin'] * xsize
                    
                    #midpoint calculation
                    midpoint_x = int(xmin1 + 0.5*(max_x-xmin1))
                    midpoint_y = int(ymin1 + 0.5*(max_y-ymin1))
                    
                    
                # box2 on rhs, box1 and box3 on lhs
                elif (row['xmin'] <= 0.5 and row['xmin3'] <= 0.5 and row['xmin2'] >= 0.5):
                    
                    # LHS: boxes on lhs
                    max_y = (max(row['ymin'], row['ymin3'])) * ysize
                    max_x = (max(row['xmin'], row['xmin3'])) * xsize
                    
                    # RHS: convert box1 coorindates
                    ymin1 = row['ymin2'] * ysize
                    xmin1 = row['xmin2'] * xsize
                    
                    #midpoint calculation
                    midpoint_x = int(xmin1 + 0.5*(max_x-xmin1))
                    midpoint_y = int(ymin1 + 0.5*(max_y-ymin1))
                    
                    
                # box3 on rhs, box1 and box2 on lhs
                elif (row['xmin'] <= 0.5 and row['xmin2'] <= 0.5 and row['xmin3'] >= 0.5):
                    
                    # LHS: boxes on lhs
                    max_y = (max(row['ymin'], row['ymin2'])) * ysize
                    max_x = (max(row['xmin'], row['xmin2'])) * xsize
                    
                    # RHS: convert box1 coorindates
                    ymin1 = row['ymin3'] * ysize
                    xmin1 = row['xmin3'] * xsize
                    
                    #midpoint calculation
                    midpoint_x = int(xmin1 + 0.5*(max_x-xmin1))
                    midpoint_y = int(ymin1 + 0.5*(max_y-ymin1))
                    
                # all boxes on lhs
                elif (row['xmin'] <= 0.5 and row['xmin2'] <= 0.5 and row['xmin3'] <= 0.5):
                    
                    # LHS: cones detected on lhs of frame
                    max_y = max(row['ymin'], row['ymin2'], row['ymin3'])
                    max_x = max(row['xmin'], row['xmin2'], row['xmin3'])
                    
                    # RHS: convert box coordinates
                    ymin1 = max_y * ysize
                    xmin1 = max_x * xsize
                        
                    #calculate midpoint between box1 and RHS of frame
                    midpoint_x = int(rhs - (0.5*(xmin1)))
                    midpoint_y = int(ymin1)
                
                
                # all boxes on rhs
                elif (row['xmin'] >= 0.5 and row['xmin2'] >= 0.5 and row['xmin3'] >=0.5):
                    
                    #calculate box closest to vehicle in horizontal direction
                    max_y = max(row['ymin'], row['ymin2'], row['ymin3'])
                    min_x = min(row['xmin'], row['xmin2'], row['xmin3'])
                    
                    #draw lines between average coords and lhs of frame
                    ymin1 = max_y * ysize
                    xmin1 = min_x * xsize
                    
                    # calculate midpoint between box and LHS of frame
                    midpoint_x = int(lhs + (0.5*(xmin1)))
                    midpoint_y = int(ymin1)
                    
            
            # draw direction vector through midpoints
            direction = cv2.arrowedLine(image, (350, 300), (midpoint_x, midpoint_y), (255,0,0), 4)
            
            
            quiver = pd.DataFrame({'x':[car_x/100], 'y':[car_y/100], 'u': [(midpoint_x)/100], 'v': [(midpoint_y)/100]})
            
            #create axb triangle using drawn line to calculate steering angle
            theta = math.degrees(math.atan(((midpoint_x - car_x)/(abs(midpoint_y - car_y)))))
            
            
            #if statement for command based on angle size
            if(theta >= -15.0 and theta <= 15.0):
                print("APP: 100%, Motor Torque: 100%, Steering: Straight "+str(theta)+" degrees")
                
                APP = 100
                Torque = 100
                Action = "Accelerate"
                Direction = "Straight"
                
            elif(theta > 15.0 and theta <= 30):
                print("APP: 80%, Motor Torque: 80%, Steering: Slight turn "+str(theta)+" degrees")
                
                APP = 80
                Torque = 80
                Action = "Slight turn"
                Direction = "Right"
            
            elif(theta < -15.0 and theta >=-30):
                print("APP: 80%, Motor Torque: 80%, Steering: Slight turn "+str(theta)+" degrees")
                
                APP = 80
                Torque = 80
                Action = "Slight turn"
                Direction = "Left"
                
            elif (theta > 30):
                print("APP: 50%, Motor Torque: 50%, Steering: Sharp turn "+str(theta)+" degrees")
                
                APP = 50
                Torque = 50
                Action = "Sharp Turn"
                Direction = "Right"
            
            else:
                print("APP: 50%, Motor Torque: 50%, Steering: Sharp turn "+str(theta)+" degrees")
                
                APP = 50
                Torque = 50
                Action = "Sharp Turn"
                Direction = "Left"
            
            
            
            command = pd.DataFrame({'Frame': [frame], 'APP /%': [APP], 'Motor Torque /%': [Torque], 'Angle /degrees':[theta], 'Action':[Action], 'Direction':[Direction] })
            
            df = pd.concat([df, command])
            df2 = pd.concat([df2, quiver])
            
            
            # rewrite image with drawn lines
            cv2.imwrite(output_directory+"frame"+"%04d" % frame+".jpg", direction)


In [ ]:
df.to_csv('feedback_acceleration_maxmin.csv', index=False)

In [ ]:
cols = df2.columns.tolist()
cols = cols[-1:] + cols[:-1]
df2 = df2[cols]
df2

In [ ]:
df2.to_csv('visualisation_output_acceleration_maxmin.csv', index=False)

In [ ]:
# sort edited images into array
img_array = []
for filename in sorted(glob.glob(output_directory+'*.jpg')):
    img = cv2.imread(filename)
    size = (xsize, ysize)
    img_array.append(img)

In [ ]:
# rewrite video of images following line drawing operation
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('acceleration_direction_vector_maxmin.mp4',fourcc, 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()